## Introduction
The SENG 474 final project required us to collect, process, and mine data from any dataset. We decided on collecting data from the video game Defence of the Ancients 2 (referred to as Dota in this report). This section provides a basic outline of the structure of a typical Dota match to aid in the understanding of the data.

Dota is a multiplayer online battle arena (MOBA) game where two teams of five unique players compete to destroy structures. Each of the ten players in a match plays as a different hero. A hero is a character with at least 4 unique abilities and there are currently 117 unique heroes in Dota. The match takes place on a square map that is bisected diagonally from northwest to southeast. The two teams, named Radiant and Dire, each control one of these sections. The Dire controls the northeastern section and the Radiant controls the southwestern section. Each team has structures on their side of the map to help defend their base from the opposing team. A Radiant and Dire’s bases are located in the southwestern and northeastern corners of the maps, respectively. A structure called the Ancient is located in the center of each team’s base. The goal of Dota is to destroy they opposing team’s Ancient. Once a team destroys the opposing team’s Ancient, the game is over and the team that destroyed their enemy’s Ancient wins.
![](http://i.imgur.com/vbFqPpt.png)

**Fig 1 Overhead view of the map.**

Each player attempts to accumulate gold and experience through a variety of means to increase the strength of their hero. A hero’s strength increases by purchasing items with gold or levelling up a hero’s abilities by accumulating a certain amount of experience. The combined wealth of a hero, calculated by adding the value of their items and the amount of gold they possess, is known as the hero’s net worth. Throughout the course of a match heroes on opposing teams will fight and kill each other. A hero is only dead temporarily and will respawn after a certain length of time. The length of time is determined by the level of the hero who was killed. The total number of kills a team accumulates is known as their score. For example, a score of 23-15 means that Radiant has 23 kills and Dire has 15 kills.

To reach the enemy’s Ancient, a team must first destroy a number of other structures on their enemy’s side of the map. In general, the more structures a team has destroyed, the closer they are to reach their enemy’s Ancient. At the end of the game the team scores are not considered when determining the winner, the only goal is the destruction of the enemy’s Ancient.
![](https://i.ytimg.com/vi/D7BFF9o3FT4/maxresdefault.jpg)
**Fig 2 View of the Radiant Ancient from a player camera perspective.**

## Data Collection
This section covers our data collection process and the difficulties we faced. We automated data collection using a python module named `collect.py`. The module uses steampowered API to get match data that is then passed to our processing module. `collect.py` uses the `GetMatchDetails` API call which returns a json object containing match statistics. The call requires two parameters, an API key and a `match_id`. The API key can be generated with a Steam account using the “Steam Community API registration page”. The match ID is unique to each game played, in order to find an initial match ID we used the website [Dotabuff]([https://www.dotabuff.com](https://www.dotabuff.com/)). To ensure we would have enough data we chose an arbitrary match’s ID from a few years ago. In total we collected more 11,000 matches over the course of a couple weeks.
```json
{
	"result": {
		"barracks_status_dire": 51,
		"barracks_status_radiant": 63,
		"cluster": 187,
		"dire_score": 75,
		"duration": 4253,
		"engine": 1,
		"first_blood_time": 37,
		"flags": 1,
		"game_mode": 22,
		"human_players": 10,
		"leagueid": 0,
		"lobby_type": 0,
		"match_id": 5000012945,
		"match_seq_num": 4193956416,
		"negative_votes": 0,
		"picks_bans": [
			{
				"hero_id": 62,
				"is_pick": false,
				"order": 0,
				"team": 0
			},
			{
				"hero_id": 1,
				"is_pick": false,
				"order": 1,
				"team": 0
			}
		],
		"players": [
			{
				"account_id": 1021944539,
				"assists": 20,
				"backpack_0": 0,
				"backpack_1": 0,
				"backpack_2": 0,
				"deaths": 16,
				"denies": 12,
				"gold_per_min": 621,
				"hero_id": 75,
				"item_0": 63,
				"item_1": 250,
				"item_2": 116,
				"item_3": 247,
				"item_4": 263,
				"item_5": 0,
				"kills": 19,
				"last_hits": 268,
				"leaver_status": 0,
				"level": 25,
				"player_slot": 0,
				"xp_per_min": 599
			},
			…
		]
	}
}
```
**Fig 3 Example of a condensed single match json object.**

The `get_match` method in `collect.py` is responsible for making the API call and collecting the json data. In this method, the initial match ID value is incremented in a loop to obtain new matches. Each of these datasets are then written to a temporary file before being moved to the `raw_data.json` file. This transfer process is necessary because the objects returned are single match results rather than a list of matches.

During our data collection process we faced challenges with the API. Initially, we were using the API call `GetMatchHistory` which returns the most recent 100 public matches as a json list. However, we quickly found that this was restricting, as we could not go further back in time than 200 matches. This meant we had to constantly ping for data in a loop while removing duplicate matches. This form of collection was far too slow and why we moved on to a different strategy.

## Pre-processing
All the data output by `collect.py` is relevant when considering a match, but for the purposes of this project some data had to be pruned. Given our understanding of the game, we tried to isolate some of the most important data points. Those being:
- Player gold
- Player experience
- Player hero
- Team score
- Team structure destruction
- Match duration
- Radiant victory status

In this project we process the data from Radiant perspective. Most of the data output by our processing module is a difference between the Radiant and Dire. We chose this format to better illustrate the performance of the Radiant team in the match.

### Filtering Raw Data
Not all of the data collected is usable. There are cases where matches were aborted during drafting phases or crashed. These cases manifest themselves as match result json objects that do not contain the win metric, `radiant_win`. Since we are mostly interested in the result of the matches, these instances cannot be used. Furthermore, there are some instances where player statistics are all `0`, this includes attributes like `hero_id`. We do not have an explanation for this behaviour but since a `hero_id` of `0` is invalid, we chose to invalidate the entire match.

A filtering method loops over the entire dataset searching for these two cases and removing them. In the end we have X% of matches that were invalid.
```python
def filter_bad_data(data):
	matches = []
	for match in data:
		if 'radiant_win' in match['result'].keys():
			for player in match['result']['players']:
				if player['hero_id'] > 0:
					matches.append(match)
					
	return matches
```
**Fig 4 Bad data filtering method.**

### Per Minute Metrics
Economy and experience are widely considered to be the most important factors for victory. As explained above, a players “net worth” is an attribute that quantifies their economic standing in a match. Experience is strongly valued as it increases a hero’s strength and is generally gained at the same rate that a player earns gold. Stangely, neither of these values are included as a part of the API response. Instead, we are given each players gold per minute and experience per minute values. Using these values, we can approximate net worth and net experience.

Since we know the duration of a match we can use these per minute metrics in a method that will output a teams total. Once the totals are calculated we subtract Radiant `net_worth` and `net_xp` from the Dire counterparts.
```python
def calculate_per_min_metrics(team_roster, duration, metric):
	metric_total = 0
	
	for player in team_roster:
		mps = float(player[metric]) / float(60)
		metric_total += (mps * duration)
		
	return metric_total
```
**Fig 5 An example of the calculation method.**

### Hero Win Rates
Before a match begins, each player has the opportunity to choose their hero from the hero pool. Naturally, some of these heroes are stronger than others. In order to quantify this, we wanted to calculate the win rate of each of the heroes.

To do this we used another steampowered API call named `GetHeroes`. This call returns a json list of the hero names and their IDs. This list is then used along with the match data by the `get_hero_winrates` method. The method iterates over each of the heroes, filters the entire dataset for matches that contained the hero, and determines whether the hero was on the winning side. Wins are tallied and divided by the number of matches, giving a percentage. The output is a dictionary with the `hero_id` and their `winrate`. This output is then used to determine each team’s average hero win rate. A difference is taken between the teams as the final output.
```python
def get_hero_winrates(data, hero_file):
	heroes = read_file(hero_file)
	winrates = {}
	for hero in heroes['result']['heroes']:
		hero_data = filter_by_hero(data, hero['id'])
		wins = 0
		for match in hero_data:
			if did_hero_win_match(match, hero['id']):
				wins += 1
						
		winrates[hero['id']] = calculate_win_percent(len(hero_data), wins)

	return winrates
```
**Fig 6 Hero win rate calculation method.**

### Structures
The purpose of the game is to destroy the enemy’s main structure. There are other structures in place, some of which being the Barracks. These structures spawn non player characters that help the destroying team push towards the enemy’s Ancient. These NPCs are valuable as they pressure opposing teams without a player having to be directly involved. Because of this Barracks are important objectives and can sway the outcome of a match.

(MAX PLS GO OVER THIS)
Barracks are represented by 8 bit unsigned integers in the json output. Starting with the sixth leftmost bit the bits represent whether a tower is standing (`1`) or has been destroyed (`0`). From the left, bits 6 and 7 each represent a tier 4 tower. The 2 leftmost bits are irrelevant. Bits 3 and 4 are bottom barracks. Bits 5 and 6 are middle barracks. Bits 7 and 8 are top barracks.
```python
def get_number_of_standing_buildings(status):
	status = bin(status)
	buildings_alive = 0
	for building in status:
		if building == '1':
			buildings_alive += 1
			
	return buildings_alive
```
**Fig 7 Structure status code.**

### Trivial Data Processing
Some of the data collected needed minimal processing. Examples of this are `score`, `duration` and `radiant_win`. Each of these existed in the `json` file on their own and were simply extracted. In the case of `score` the opposing teams scores were subtracted from one another.

### Final Output
The final output of the data processing module are two `.csv` files. One containing 100 rows of test data and the other containing over 11,000 rows of test data. These files are created by `build_csv.ipynb` using the `process.py` module.
```
net_worth | net_xp | team_winrate_avg | score | net_barracks | duration | radiant_win
----------+--------+------------------+-------+--------------+----------+------------
16799.35  | -70.88 |       -2.43      |   -3  |       2      |   4253   |    True
```
**Fig 8 Example of the csv output.**

## Visualization
To help visualize some of the we have amassed here are some swell graphs.